In [ ]:
from gs_quant.session import Environment, GsSession
from gs_quant.instrument import IRSwaption
from gs_quant.risk import CurveScenario, MarketDataPattern
from gs_quant.markets import MarketDataCoordinate
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

In [ ]:
swaption = IRSwaption('Receive', '5y', 'USD', expiration_date='13m', strike='atm')
swaption.resolve()

In [ ]:
original_price = swaption.price()
market_data = swaption.market().market_data_dict
print('Base price:     {:,.2f}'.format(original_price))

In [ ]:
# Price the swaption under a curve move of 10bp parallel shift
parallel_shift_scenario = CurveScenario(market_data_pattern=MarketDataPattern('IR', 'USD'), parallel_shift=10)

with parallel_shift_scenario:
    swaption_parallel_shift = swaption.price()
    market_data_parallel_shift = swaption.market().market_data_dict
print('Price under parallel shift: {:,.2f}'.format(swaption_parallel_shift))

In [ ]:
# Compare swap rate market data coordinates before and after curve scenario shock
coord = MarketDataCoordinate(mkt_type="IR",mkt_asset="USD")
market_data_df = pd.DataFrame([{mkt_data.coordinate: mkt_data.value * 1e4 for mkt_data in market_data if (mkt_data.coordinate.mkt_type=="IR" and mkt_data.coordinate.mkt_class=="SWAP")},
                               {mkt_data.coordinate: mkt_data.value * 1e4 for mkt_data in market_data_parallel_shift if (mkt_data.coordinate.mkt_type=="IR" and mkt_data.coordinate.mkt_class=="SWAP")}],
                              index=['Values', 'Shocked values']).transpose()
market_data_df

In [ ]:
# Plotting swap rate market data before and after curve scenario shock 
swap_curve = pd.DataFrame.from_dict({int(''.join(list(filter(str.isdigit, str(v))))): market_data_df.loc[v]
                                    for v in market_data_df.index}, orient='index')

swap_curve['Shock'] = swap_curve['Shocked values'] - swap_curve['Values']
swap_curve.plot(figsize=(12, 8), title='USD Swap Curve Before and After Parallel Shock')
plt.xlabel('Tenor (years)')
plt.ylabel('bp')